In [ ]:
#------------------------------------------------------
# 1. Setup and Installs
#------------------------------------------------------
! pip install ultralytics -q # Install YOLOv8 library

import os
import shutil
import random
import yaml
from ultralytics import YOLO
from IPython.display import display, Image # To display results later

print("Setup Complete.")

#------------------------------------------------------
# 2. Define Paths and Parameters
#------------------------------------------------------
# Input dataset path on Kaggle
INPUT_DATASET_PATH = "/kaggle/input/classification/dataset1/"
RIPE_IMG_DIR = os.path.join(INPUT_DATASET_PATH, "Ripe")

# Output directory for prepared YOLO dataset (in Kaggle's writable directory)
OUTPUT_YOLO_DATA_DIR = "/kaggle/working/mango_yolo_dataset/"

# YOLO dataset structure paths
IMG_TRAIN_DIR = os.path.join(OUTPUT_YOLO_DATA_DIR, "images/train")
LBL_TRAIN_DIR = os.path.join(OUTPUT_YOLO_DATA_DIR, "labels/train")
IMG_VAL_DIR = os.path.join(OUTPUT_YOLO_DATA_DIR, "images/val")
LBL_VAL_DIR = os.path.join(OUTPUT_YOLO_DATA_DIR, "labels/val")

# Parameters
VAL_SPLIT = 0.20 # 20% validation split
CLASS_NAME = "Ripe"
CLASS_ID = 0 # YOLO class IDs are 0-indexed

# Training Hyperparameters (adjust as needed)
EPOCHS = 30 # Start with a moderate number
IMG_SIZE = 640 # Standard YOLOv8 input size
BATCH_SIZE = 16 # Adjust based on GPU memory (16 is often a good start)
MODEL_NAME = 'yolov8n.pt' # Nano version - fast, less accurate
PROJECT_NAME = '/kaggle/working/yolo_training_results'
RUN_NAME = 'mango_ripe_only_run'

#------------------------------------------------------
# 3. Prepare Dataset in YOLO Format
#------------------------------------------------------
print("Preparing dataset in YOLO format...")

# Create necessary directories
os.makedirs(IMG_TRAIN_DIR, exist_ok=True)
os.makedirs(LBL_TRAIN_DIR, exist_ok=True)
os.makedirs(IMG_VAL_DIR, exist_ok=True)
os.makedirs(LBL_VAL_DIR, exist_ok=True)

# Get list of ripe images
ripe_images = [f for f in os.listdir(RIPE_IMG_DIR) if os.path.isfile(os.path.join(RIPE_IMG_DIR, f))]
random.shuffle(ripe_images) # Shuffle for random split

# Split into train and validation sets
split_idx = int(len(ripe_images) * (1 - VAL_SPLIT))
train_images = ripe_images[:split_idx]
val_images = ripe_images[split_idx:]

print(f"Total 'Ripe' images: {len(ripe_images)}")
print(f"Training images: {len(train_images)}")
print(f"Validation images: {len(val_images)}")

# --- Function to create pseudo-annotation (whole image) ---
def create_pseudo_label(image_filename, label_dir):
    """
    Creates a YOLO label file assuming the object covers the entire image.
    Format: <class_id> <x_center> <y_center> <width> <height> (normalized)
    """
    label_filename = os.path.splitext(image_filename)[0] + ".txt"
    label_path = os.path.join(label_dir, label_filename)
    with open(label_path, 'w') as f:
        # Class 0, center (0.5, 0.5), width 1.0, height 1.0
        f.write(f"{CLASS_ID} 0.5 0.5 1.0 1.0\n")

# --- Process Training Images ---
print("Processing Training Set...")
for img_name in train_images:
    # Copy image
    shutil.copy(os.path.join(RIPE_IMG_DIR, img_name), os.path.join(IMG_TRAIN_DIR, img_name))
    # Create pseudo-label
    create_pseudo_label(img_name, LBL_TRAIN_DIR)

# --- Process Validation Images ---
print("Processing Validation Set...")
for img_name in val_images:
    # Copy image
    shutil.copy(os.path.join(RIPE_IMG_DIR, img_name), os.path.join(IMG_VAL_DIR, img_name))
    # Create pseudo-label
    create_pseudo_label(img_name, LBL_VAL_DIR)

print("Dataset preparation finished.")
print("--- IMPORTANT NOTE ---")
print("Generated labels assume the *entire image* is the 'Ripe' mango.")
print("For true object detection, bounding box annotations are required.")
print("----------------------")


#------------------------------------------------------
# 4. Create data.yaml file for YOLO
#------------------------------------------------------
yaml_data = {
    'train': IMG_TRAIN_DIR,
    'val': IMG_VAL_DIR,
    'nc': 1, # Number of classes
    'names': [CLASS_NAME] # List of class names
}

yaml_file_path = os.path.join(OUTPUT_YOLO_DATA_DIR, "data.yaml")

with open(yaml_file_path, 'w') as f:
    yaml.dump(yaml_data, f, default_flow_style=False)

print(f"data.yaml created at: {yaml_file_path}")
# You can print the content to verify
# !cat {yaml_file_path}

#------------------------------------------------------
# 5. Train the YOLOv8 Model
#------------------------------------------------------
print("Starting YOLOv8 training...")

# Load a pretrained YOLOv8n model
model = YOLO(MODEL_NAME)

# Train the model
results = model.train(
    data=yaml_file_path,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    project=PROJECT_NAME,
    name=RUN_NAME,
    # patience=10, # Optional: Early stopping patience
    # device=0 # Optional: Specify GPU device (0 usually works in Kaggle)
)

print("Training finished.")
print(f"Results saved to: {PROJECT_NAME}/{RUN_NAME}")

#------------------------------------------------------
# 6. (Optional) View Results / Example Prediction
#------------------------------------------------------
# You can find the best model weights at:
# /kaggle/working/yolo_training_results/mango_ripe_only_run/weights/best.pt

# Example of loading the trained model and predicting on a validation image
print("\nExample Prediction:")
trained_model_path = os.path.join(PROJECT_NAME, RUN_NAME, 'weights/best.pt')
if os.path.exists(trained_model_path) and val_images:
    # Load the trained model
    trained_model = YOLO(trained_model_path)

    # Select a random validation image
    sample_image_name = random.choice(val_images)
    sample_image_path = os.path.join(IMG_VAL_DIR, sample_image_name)

    print(f"Predicting on: {sample_image_path}")
    # Run prediction
    predict_results = trained_model.predict(source=sample_image_path, save=True) # save=True saves the image with boxes

    # The predicted image will be saved in /kaggle/working/runs/detect/predict...
    # Display the saved image (find the latest prediction folder)
    try:
        predict_dir = sorted([d for d in os.listdir('/kaggle/working/runs/detect') if d.startswith('predict')], reverse=True)[0]
        output_image_path = os.path.join('/kaggle/working/runs/detect', predict_dir, sample_image_name)
        if os.path.exists(output_image_path):
            print("Displaying prediction result:")
            display(Image(filename=output_image_path))
        else:
             print(f"Could not find predicted image at expected location: {output_image_path}")
    except Exception as e:
        print(f"Could not display prediction image: {e}")

else:
    print("Could not find trained model or validation images for prediction example.")

In [ ]:
# [Your previous YOLOv8 training code should be above this]
# Ensure the following variables are defined from your training script:
# - results: The object returned by model.train()
# - PROJECT_NAME: e.g., '/kaggle/working/yolo_training_results'
# - RUN_NAME: e.g., 'mango_ripe_only_run'
# - EPOCHS: The number of epochs used for training
# - IMG_SIZE: The image size used for training (e.g., 640)
# - MODEL_NAME: The base model used (e.g., 'yolov8n.pt')

# Define the path to the best model reliably
# best_model_path = os.path.join(PROJECT_NAME, RUN_NAME, 'weights/best.pt')

# # ------------------------------------
# # 7. Save Model to Hugging Face Hub
# # ------------------------------------
# print("\nAttempting to upload model to Hugging Face Hub...")

# # First, check if the best model file actually exists
# if not os.path.exists(best_model_path):
#     print(f"❌ Error: Best model not found at {best_model_path}. Skipping Hugging Face upload.")
#     print("This might happen if the training failed or did not produce 'best.pt'.")
# else:
#     try:
#         # Install the huggingface_hub package if not already installed
#         !pip install huggingface_hub -q

#         # Import necessary packages
#         from huggingface_hub import HfApi, login
#         import shutil
#         import datetime # To get the current date for the README

#         # --- IMPORTANT: SECURITY RECOMMENDATION ---
#         # Avoid hardcoding tokens. Use Kaggle Secrets for better security.
#         # 1. Go to "Add-ons" -> "Secrets" in your Kaggle notebook.
#         # 2. Add a secret named "HF_TOKEN" with your Hugging Face write token as the value.
#         # 3. Uncomment the following lines to use the secret:
#         # from kaggle_secrets import UserSecretsClient
#         # user_secrets = UserSecretsClient()
#         # hf_token = user_secrets.get_secret("HF_TOKEN")
#         # print("Using Hugging Face token from Kaggle Secrets.")
#         # --- For this example, using the provided token (less secure) ---
#         hf_token = "your_tokenhf_NWCRwmEBVFlNmHjomfgdZwnTOSvPYAfGWU" # Replace with your actual token or use Secrets
#         if hf_token == "your_token": # Basic check if default token is used
#              print("⚠️ WARNING: Using a hardcoded Hugging Face token. Consider using Kaggle Secrets.")

#         print("Logging into Hugging Face Hub...")
#         # add_to_git_credential=False is often helpful in non-interactive environments like Kaggle
#         login(token=hf_token, add_to_git_credential=False)
#         print("Login successful.")

#         # Define the repository name and create a local directory for upload files
#         repo_id = "sravan27745/yolo_mangop" # Your HF username / desired repo name
#         # Use /kaggle/working/ as it's the writable directory in Kaggle
#         local_dir = "/kaggle/working/hf_model_upload"

#         # Create the temporary directory for the files to upload
#         os.makedirs(local_dir, exist_ok=True)
#         print(f"Created local directory for upload: {local_dir}")

#         # Copy the best model weights to the upload directory
#         shutil.copy(best_model_path, os.path.join(local_dir, "best.pt"))
#         print(f"Copied best model weights ({best_model_path}) to upload directory.")

#         # --- Create a README.md file with relevant model information ---
#         readme_path = os.path.join(local_dir, "README.md")
#         print(f"Creating {readme_path}...")

#         # Extract base model name (e.g., 'yolov8n' from 'yolov8n.pt')
#         base_model_name = MODEL_NAME.replace('.pt', '')
#         training_date = datetime.datetime.now().strftime("%Y-%m-%d") # Get current date

#         # Class name comes from the training script
#         trained_class_name = CLASS_NAME # Should be 'Ripe' based on previous code

#         readme_content = f"""---
# language: en
# license: mit # Or choose another appropriate license
# library_name: ultralytics
# tags:
# - yolov8
# - object-detection
# - computer-vision
# - mangoes
# - agriculture
# - ripe-mango
# datasets:
# - custom-mango-classification # Describe your dataset source if possible
# pipeline_tag: object-detection
# widget:
# - src: https://ultralytics.com/images/bus.jpg # Example image for widget
#   example_title: YOLOv8 Ripe Mango Detection Example
# ---

# # YOLOv8 Ripe Mango Detection Model (Classification-based)

# This model was trained using the Ultralytics YOLOv8 library to detect **ripe mangoes**.

# ## Model Details
# - **Base model:** `{base_model_name}`
# - **Class detected:** '{trained_class_name}' (Class ID 0)
# - **Epochs trained:** `{EPOCHS}`
# - **Input size:** `{IMG_SIZE}x{IMG_SIZE}`
# - **Training date:** `{training_date}`

# ## Training Data & Approach
# This model was trained on a dataset originally structured for image classification (folders: OverRipe, Ripe, UnRipe). Only images from the **'{trained_class_name}'** category were used for training this specific detection model.

# **Important Note:** During training, pseudo-labels were generated assuming the ripe mango object covers the *entire* image frame (YOLO format: `0 0.5 0.5 1.0 1.0`). Therefore, while it uses a detection framework, its primary function based on this training is closer to classifying if an image contains a ripe mango. For precise bounding box localization *within* an image containing multiple objects or backgrounds, fine-tuning with accurate bounding box annotations is necessary.

# ## Usage
# Make sure you have `ultralytics` and `torch` installed:
# ```bash
# pip install ultralytics torch

In [ ]:
# # ------------------------------------
# # 1. Install/Ensure Libraries
# # ------------------------------------
# # Need ultralytics for YOLO and gradio for the interface
# # Also ensure transformers and huggingface_hub are available for HF integration
# !pip install ultralytics gradio transformers huggingface_hub torch -q # Ensure torch is also there
# print("Ultralytics, Gradio, Transformers, Hub, and Torch libraries installed/updated.")

# # ------------------------------------
# # 2. Import Libraries
# # ------------------------------------
# import os
# import gradio as gr
# from ultralytics import YOLO
# from PIL import Image
# import numpy as np # Use standard 'np' alias
# import glob # To find example images if needed
# import torch # Import torch explicitly for potential device checks later

# print("Libraries imported.")

# # ------------------------------------
# # 3. Define Hugging Face Model ID
# # ------------------------------------
# # Your Hugging Face repository ID (username/repo_name)
# hf_model_id = "sravan27745/yolo_mangop"
# print(f"Using Hugging Face Hub model repository ID: {hf_model_id}")

# # ------------------------------------
# # 4. Load the YOLO Model from Hugging Face
# # ------------------------------------
# print(f"Attempting to load YOLOv8 model directly from Hugging Face Hub repo: {hf_model_id}...")
# # Ultralytics' YOLO class is often smart enough to find 'best.pt' or other standard weights
# # within the specified repository automatically.
# try:
#     # Load the model using just the repo ID
#     model = YOLO(hf_model_id)
#     print(f"YOLOv8 model loaded successfully from {hf_model_id}.")
#     # Optional: Print model info (helpful for debugging)
#     model.info()
#     # You can check the specific class name loaded
#     print(f"Model trained for classes: {model.names}")
#     if 0 in model.names:
#          detected_class_name = model.names[0] # Assuming 'Ripe' is class 0
#          print(f"Assuming Class ID 0 corresponds to '{detected_class_name}' based on training.")
#     else:
#          detected_class_name = "Detected Object" # Fallback name
#          print("Warning: Could not automatically determine the name for Class ID 0.")

# except Exception as e:
#     print(f"❌ Error loading model from Hugging Face repo '{hf_model_id}': {e}")
#     print("\nTroubleshooting:")
#     print(f"  1. Ensure the repository '{hf_model_id}' exists and is PUBLIC on Hugging Face Hub.")
#     print(f"  2. Ensure a valid YOLOv8 model file (like 'best.pt') exists at the root of that repository.")
#     print(f"  3. Check your internet connection from the Kaggle kernel.")
#     print(f"  4. As a fallback, you can try specifying the file explicitly: model = YOLO('{hf_model_id}/best.pt')")
#     # Stop execution if model loading fails
#     raise RuntimeError(f"Failed to load model {hf_model_id}") from e

# # ------------------------------------
# # 5. Define Inference Function
# # ------------------------------------
# def detect_ripe_mangoes(input_image: Image.Image, confidence_threshold: float = 0.25):
#     """
#     Performs inference on the input image using the loaded YOLOv8 model
#     to detect the 'Ripe' mango class (as trained).

#     Args:
#         input_image (PIL.Image.Image): Image uploaded by the user.
#         confidence_threshold (float): Minimum confidence score for detections.

#     Returns:
#         tuple(PIL.Image.Image, str): Annotated image and status message.
#     """
#     if input_image is None:
#         # Return a placeholder or blank image if needed by Gradio output component
#         # Create a small blank image
#         blank_image = Image.new('RGB', (200, 100), color = 'grey')
#         # You could potentially draw text on it too if desired
#         return blank_image, "Status: Please upload an image."

#     print(f"Received image for inference. Type: {type(input_image)}")
#     print(f"Running inference with confidence threshold: {confidence_threshold}")

#     # Perform inference using the loaded 'model'
#     # device=0 forces GPU if available, remove or set to 'cpu' if needed
#     try:
#         # Setting verbose=False to reduce console spam during prediction
#         results = model.predict(source=input_image, conf=confidence_threshold, device=None, save=False, verbose=False) # Use None for auto-device selection
#     except Exception as pred_e:
#         print(f"Error during model.predict: {pred_e}")
#         # Return original image and error message
#         return input_image, f"Status: Error during prediction: {pred_e}"


#     if not results:
#         print("Warning: model.predict returned None or empty list.")
#         return input_image, "Status: Prediction returned no results."

#     # Assuming prediction on a single image, get the first result object
#     result = results[0]

#     # Get the annotated image as a NumPy array (BGR format by default)
#     # Plotting includes boxes, labels, and confidence scores
#     annotated_image_np = result.plot(conf=True) # Set conf=True to show confidence on boxes

#     # Convert NumPy array (BGR) back to PIL Image (RGB) for Gradio display
#     annotated_image_pil = Image.fromarray(annotated_image_np[..., ::-1]) # Correct BGR to RGB conversion

#     num_detections = len(result.boxes)
#     class_name_display = detected_class_name # Use the name determined during model load

#     if num_detections > 0:
#          # Optional: Get confidences if needed
#          # confs = result.boxes.conf.tolist()
#          status_message = f"Status: Detected {num_detections} instance(s) of '{class_name_display}'."
#     else:
#          status_message = f"Status: No '{class_name_display}' detected with confidence >= {confidence_threshold:.2f}."

#     print(status_message) # Log status to notebook console

#     return annotated_image_pil, status_message

# # ------------------------------------
# # 6. Find Example Images (Optional - Requires Dataset Input)
# # ------------------------------------
# # --- IMPORTANT ---
# # This section requires the original dataset (or at least some sample images)
# # to be attached to the Kaggle notebook's input.
# # Update the path '/kaggle/input/your_dataset_name/path/to/images' accordingly.
# # If the dataset isn't attached, this will be skipped gracefully.
# # -----------------
# example_image_dir = '/kaggle/input/classification/dataset1/Ripe' # ADJUST THIS PATH if needed! (Using Ripe folder from original structure)
# example_images = []

# print(f"\nLooking for example images in: {example_image_dir}")
# if os.path.isdir(example_image_dir):
#     image_patterns = ['*.jpg', '*.jpeg', '*.png', '*.webp'] # Add common image types
#     found_files = []
#     for pattern in image_patterns:
#          # Use recursive=True if images might be in subdirectories
#          found_files.extend(glob.glob(os.path.join(example_image_dir, pattern))) # Removed recursive for simplicity

#     # Remove duplicates and limit the number of examples
#     example_images = sorted(list(set(found_files)))[:5] # Get up to 5 examples

#     if example_images:
#         print(f"Found {len(example_images)} example images:")
#         for ex_img in example_images:
#              print(f"  - {os.path.basename(ex_img)}") # Print only filename for brevity
#     else:
#         print("Found the directory, but no image files matching patterns.")
# else:
#     print(f"Example image directory not found or inaccessible: {example_image_dir}")
#     print("Ensure the dataset is correctly attached to the notebook under '/kaggle/input/'.")
#     print("Examples section will be skipped.")


# # ------------------------------------
# # 7. Create and Launch Gradio Interface
# # ------------------------------------
# print("\nSetting up Gradio interface...")

# # Use gr.Blocks for more layout control
# with gr.Blocks(theme=gr.themes.Soft()) as demo: # Added a theme for nicer look
#     gr.Markdown(f"""
#     # YOLOv8 'Ripe' Mango Detector 🥭
#     Detects the '{detected_class_name}' class using a model fine-tuned from YOLOv8.
#     Model loaded from Hugging Face Hub: `{hf_model_id}`.
#     **Note:** This model was trained assuming the ripe mango fills most of the image (using classification-style data). Performance may vary on complex scenes.
#     """)

#     with gr.Row():
#         with gr.Column(scale=1):
#             input_img = gr.Image(type="pil", label="Upload Mango Image")
#             conf_slider = gr.Slider(minimum=0.10, maximum=0.95, value=0.25, step=0.05, label="Confidence Threshold")
#             submit_btn = gr.Button("🔍 Detect Mangoes", variant="primary") # Make button stand out

#         with gr.Column(scale=1):
#             output_img = gr.Image(type="pil", label="Output Image with Detections")
#             status_text = gr.Textbox(label="Detection Status", interactive=False) # To show detection count/messages

#     if example_images:
#         print("Creating Gradio examples...")
#         gr.Examples(
#             examples=example_images,
#             inputs=input_img, # The component where examples are loaded
#             outputs=[output_img, status_text], # Components to display results in
#             fn=detect_ripe_mangoes, # Function to call for examples
#             # Cache results for examples to speed up loading after first click
#             # Be cautious with caching if your function has side effects or uses external state that changes
#             cache_examples=False, # Set to True if function is pure and dataset is static
#             # Define how many examples per page, adjust as needed
#             examples_per_page=5
#         )
#     else:
#          gr.Markdown("_(Example images could not be loaded. Attach the dataset to enable examples.)_")

#     # Connect the button click to the inference function
#     submit_btn.click(
#         fn=detect_ripe_mangoes,
#         inputs=[input_img, conf_slider],
#         outputs=[output_img, status_text]
#     )

# print("\nLaunching Gradio interface...")
# print("Please wait for the public URL to appear below...")
# # Launch the interface.
# # share=True generates a public link (essential for running in Kaggle/Colab)
# # debug=True provides logs in the notebook output, helpful for troubleshooting
# # inline=False opens the interface in a new tab (often better for layout)
# demo.launch(share=True, debug=True, inline=False)

# print("\n----------------------------------------------------------------------")
# print("Gradio Setup Complete. If successful, a public URL should be visible above.")
# print("If the interface doesn't load, check the output logs for errors.")
# print("Remember to STOP the Kaggle kernel when you are finished.")
# print("----------------------------------------------------------------------")